In [32]:
import pymongo
import pickle
import pandas as pd
import numpy as np
from __future__ import unicode_literals 
import nltk
import re
from bosonnlp import BosonNLP
nlp = BosonNLP('Yy7Tpb83.27824.Zdrfgp6JtKTf')

In [5]:
db.grant.find({"patentNumber":"9532494"},{"patentNumber":1, "abstractText":1})

In [13]:
myclient = pymongo.MongoClient("mongodb://140.112.106.212:27017/")

In [16]:
dblist = myclient.list_database_names()
if "us" in dblist:
    mydb = myclient["us"]

In [19]:
collist = mydb.list_collection_names()
if "grant" in collist:
    mycol = mydb["grant"]

In [33]:
patents = []

In [36]:
for x in mycol.find({},{"patentNumber":1, "abstractText":1}):
    patents.append([x['patentNumber'],x['abstractText']])
print(len(patents))

97531


In [46]:
with open('eng_patents.pickle','wb') as f:
    pickle.dump(patents,f)

In [2]:
with open('eng_patents.pickle','rb') as f:
    patents = pickle.load(f)

In [4]:
def en_regex(a):
    #a = re.sub('[a-zA-Z]','',a)
    a = re.sub('[{}、「」()：；！？，。－"\']','',a)
    a = re.sub(',.:;!?',' ',a)
    a = re.sub('\n','',a)
    return a

In [33]:
def regex(a):
    a = re.sub('[,、：；！？，。－"\']','',a)
    a = re.sub('[「」(){}:;!?]','',a)
    a = re.sub('\n','',a)
    #a = re.sub('[(),{}\u3000\*\|=-\[\]\n.]','',a)
    a = re.sub(u"\\（.*?\\）|\\『.*?』|\\「.*?」|\\〔.*?〕|\\[.*?]|\\〈.*?〉", '', a)
    return a

In [5]:
sents = []
for i in range(0,len(patents)):
    tmp = patents[i][1]
    tmp = tmp.split('.')
    for line in tmp:
        if not line == '':
            sents.append(en_regex(line.strip().lower()))
print(len(sents))

352253


In [69]:
eng = []
for x in range(0,len(sents)):
    if x%10000==0:
        print(x)
    tmpstr = '<bos>/<sos>'
    words = nltk.word_tokenize(sents[x])
    word_tag = nltk.pos_tag(words)
    for i in range(0,len(word_tag)):
        tmpstr = tmpstr+' '+word_tag[i][0]+'/'+word_tag[i][1]
    tmpstr = tmpstr+' <eos>/<sos>'
    eng.append(tmpstr)
print(len(eng))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
352253


In [70]:
eng[51515]

'<bos>/<sos> methods/NNS and/CC devices/NNS for/IN biological/JJ sample/NN preparation/NN and/CC analysis/NN are/VBP disclosed/VBN <eos>/<sos>'

In [71]:
with open('0503英文專利資料.txt', 'w+') as f:
    for item in eng:
        f.write("%s\n" % item)

In [74]:
with open('英文專利單句.txt', 'w+') as f:
    for item in sents:
        f.write("%s\n" % item)

In [10]:
import os
tmp = []
x = 1
for i in range(1,len(sents)):
    tmp.append(sents[i-1])
    if i%100==0 or i==len(sents):
        with open('EngPatentSents/epsents'+str(x)+'.txt', 'w+') as f:
            for item in tmp:
                f.write("%s\n" % item)
            tmp = []
            x = x+1

In [21]:
testen = []
testch = []
tmp = []
tmp2 = []
for i in range(1,100):
    with open('0505/epsents'+str(i)+'.txt', 'r') as f:
        for line in f:
            tmp.append(line)
    with open('0505/'+str(i)+'.txt', 'r') as f:
        for line in f:
            tmp2.append(line)
    if len(tmp) == len(tmp2):
        for line in tmp:
            testen.append(line)
        for line in tmp2:
            testch.append(line)
        tmp = []
        tmp2 = []

In [22]:
print(len(testen),len(testch))

9900 9900


In [37]:
outputs = []
for i in range(0,len(testen)):
    if len(testen[i].split()) <=20 and len(testen[i].split()) >= 10:
        outputs.append([en_regex(testen[i]),regex(testch[i])])
len(outputs)

3389

In [38]:
outputs

['each blade includes a blade processor and a plurality of smart couplers',
 '每個刀片包括刀片處理器和多個智能耦合器']

In [39]:
cboson = []
tag_boson = []
tmp = ''
start = 0
for i in range(0,len(outputs)):
    try:
        result = nlp.tag(outputs[i][1])
        for word in result[0]['word']:
            tmp = tmp+word+'#'
        cboson.append(tmp)
        tmp = ''
        for tag in result[0]['tag']:
            tmp = tmp+tag+'#'
        tag_boson.append(tmp)
        tmp = ''
    except:
        print('summary',i,'does not work.')
    start = start+1
    if(i%100==0):
        print('finish',start,'sentences')
        #print(result)

finish 1 sentences
finish 101 sentences
finish 201 sentences
finish 301 sentences
finish 401 sentences
finish 501 sentences
finish 601 sentences
finish 701 sentences
finish 801 sentences
finish 901 sentences
finish 1001 sentences
finish 1101 sentences
finish 1201 sentences
finish 1301 sentences
finish 1401 sentences
finish 1501 sentences
finish 1601 sentences
finish 1701 sentences
finish 1801 sentences
finish 1901 sentences
finish 2001 sentences
finish 2101 sentences
finish 2201 sentences
finish 2301 sentences
finish 2401 sentences
finish 2501 sentences
finish 2601 sentences
finish 2701 sentences
finish 2801 sentences
finish 2901 sentences
finish 3001 sentences
finish 3101 sentences
finish 3201 sentences
finish 3301 sentences


In [40]:
engs = []
for x in range(0,len(outputs)):
    if x%1000==0:
        print(x)
    tmpstr = '<bos>/<sos>'
    words = nltk.word_tokenize(outputs[x][0])
    word_tag = nltk.pos_tag(words)
    for i in range(0,len(word_tag)):
        tmpstr = tmpstr+' '+word_tag[i][0]+'/'+word_tag[i][1]
    tmpstr = tmpstr+' <eos>/<sos>'
    engs.append(tmpstr)

0
1000
2000
3000


In [41]:
chi = []
for x in range(0,len(cboson)):
    a = cboson[x].split('#')
    b = tag_boson[x].split('#')
    if(len(a)-len(b))==0:
        tmpstr = '<bos>/<sos>'
        for i in range(0,len(a)-1):
            tmpstr = tmpstr+' '+a[i]+'/'+b[i]
        tmpstr = tmpstr+' <eos>/<sos>'
    else:
        tmpstr = '<bos>/<sos>''<eos>/<sos>'
        print(x,'Wtf')
    chi.append(tmpstr)

In [42]:
with open('0505英文直接對應文本.txt', 'w+') as f:
    for item in engs:
        f.write("%s\n" % item)
        
with open('0505中文直接對應文本.txt', 'w+') as f:
    for item in chi:
        f.write("%s\n" % item)